# Tests of simqso

This notebook is a sandbox notebook for testing various aspects of simqso.

In [ ]:
from desisim.templates import SIMQSO
qq = SIMQSO()

In [ ]:
%time flux1, wave1, meta1, qmeta1 = qq.make_templates(5, seed=555, return_qsometa=True)

In [ ]:
%time flux2, wave2, meta2, qmeta2 = qq.make_templates(input_meta=meta1, input_qsometa=qmeta1, return_qsometa=True)

In [ ]:
meta1

In [ ]:
meta2

In [ ]:
qmeta1

In [ ]:
qmeta2

In [ ]:
stop

In [ ]:
from desisim.test.test_templates import TestTemplates

In [ ]:
tt = TestTemplates()
tt.setUp()

In [ ]:
tt.test_input_meta()

In [ ]:
stop

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from astropy import cosmology
from simqso.sqgrids import *
from simqso import sqbase
from simqso.sqrun import buildSpectraBulk,buildQsoSpectrum,save_spectra,load_spectra
from simqso.sqmodels import BOSS_DR9_PLEpivot,get_BossDr9_model_vars

In [4]:
def doit(nqso=3, seed=1, input_qsos=None):

    rand = np.random.RandomState(seed)
    wave = sqbase.fixed_R_dispersion(500, 20e4, 1000)

    cosmo = cosmology.core.FlatLambdaCDM(70.0, 0.3)
    kcorr = sqbase.ContinuumKCorr('DECam-r', 1450, effWaveBand='SDSS-r')
    
    if input_qsos is None:
        zin = rand.uniform(2, 4, nqso)
        qsos = generateQlfPoints(BOSS_DR9_PLEpivot(cosmo=cosmo),
                                 (17, 22), (2.0, 4.0),
                                 kcorr=kcorr, zin=zin,
                                 qlfseed=seed, gridseed=seed)
        qsos.write(outFn='test.fits')
    else:
        M = AbsMagVar(FixedSampler(input_qsos.absMag), kcorr.restBand)
        m = AppMagVar(FixedSampler(input_qsos.appMag), kcorr.obsBand)
        z = RedshiftVar(FixedSampler(input_qsos.z))
        qsos = QsoSimPoints([M, m, z], cosmo=cosmo, units='flux', seed=seed)

    sedVars = get_BossDr9_model_vars(qsos, wave, noforest=False)
    qsos.addVars(sedVars)
    qsos.loadPhotoMap([('DECam', 'DECaLS'), ('WISE', 'AllWISE')])

    _, spectra = buildSpectraBulk(wave, qsos, saveSpectra=True, maxIter=5, verbose=10)

    return spectra, wave, qsos

In [5]:
spec1, wave, qsos = doit(seed=555)

simulating  3  quasar spectra
units are  flux
buildSpectra iteration  1  out of  5
--> delta mag mean = -0.2013750, rms = 0.0482204, |max| = 0.2377449
buildSpectra iteration  2  out of  5
--> delta mag mean = -0.0224523, rms = 0.0146702, |max| = 0.0361112
buildSpectra iteration  3  out of  5
--> delta mag mean = -0.0007557, rms = 0.0005813, |max| = 0.0014222


In [ ]:
spec2, _, qsos2 = doit(seed=555, input_qsos=qsos)
np.all(spec1 == spec2)

In [ ]:
qsos.data

In [ ]:
qsos2.data

In [ ]:
plt.plot(wave, spec1[0, :])
plt.plot(wave, spec2[0, :], alpha=0.5)
plt.xlim(3000, 8000)

In [ ]:
plt.plot(wave, spec1[0, :] / spec2[0, :])
plt.xlim(3000, 8000)

In [ ]:
stop

In [ ]:
spec1, wave, qsos = doit(seed=1)
spec2, _, _ = doit(seed=1)
spec3, _, _ = doit(seed=2)
spec4, _, _ = doit(seed=1)

In [ ]:
assert(np.all(spec1 == spec2))

In [ ]:
assert(np.all(spec3 != spec2))

In [ ]:
assert(np.all(spec4 == spec1))